**Muhammad Faris Akbar**
<br></br>
**Fundamental Deep Learning - Sentimen Analisis Shopee**

In [3]:
import sys
import os

try:
    # Jika dijalankan sebagai file .py
    base_path = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Jika dijalankan di Jupyter/Interactive
    base_path = os.getcwd()

parent_dir = os.path.abspath(os.path.join(base_path, '..'))
sys.path.append(parent_dir)

print("Base Path:", base_path)
print("Parent Directory:", parent_dir)

Base Path: e:\Github\Dicoding-Submission-FDL\Analisis Sentimen
Parent Directory: e:\Github\Dicoding-Submission-FDL


In [5]:
from google_play_scraper import reviews, Sort
import pandas as pd
from Helper import *
from transformers import pipeline
from tqdm import tqdm

e:\Github\Dicoding-Submission-FDL\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


**Function Scraping Data Google Play**

In [ ]:
def scrape_google_play(app_id, total_count, country='id', lang='id'):
    """
    Fungsi untuk melakukan scraping ulasan dari Google Play Store.
    
    app_id (str): ID aplikasi di Play Store (contoh: 'com.shopee.id')
    total_count (int): Target jumlah data yang ingin diambil
    country (str): Kode negara (default Indonesia 'id')
    lang (str): Bahasa ulasan (default Indonesia 'id')
    """
    print(f"Proses scraping untuk aplikasi: {app_id}...")
    
    result, continuation_token = reviews(
        app_id,
        lang=lang, 
        country=country,
        sort=Sort.NEWEST, # Sort.NEWEST digunakan untuk mendapat data terbaru
        count=total_count, 
        filter_score_with=None # Mengambil semua rating (1-5)
    )

    print(f"Berhasil mengambil {len(result)} data mentah.")
    
    # Konversi hasil scraping ke dalam DataFrame (Tabel)
    df = pd.DataFrame(result)
    
    return df

def label_with_indobert(df):
    pretrained_name = "w11wo/indonesian-roberta-base-sentiment-classifier"
    
    nlp = pipeline(
        "sentiment-analysis", 
        model=pretrained_name, 
        tokenizer=pretrained_name,
        truncation=True, # Potong teks jika terlalu panjang (>512 kata)
        max_length=512
    )
   
    labels = []
    scores = []
    
    for text in tqdm(df['content']):
        try:
            # Prediksi sentimen
            result = nlp(text)[0] 
            
            # Mapping hasil model ke format yang kita inginkan (Kapitalisasi awal)
            label = result['label'] # output: 'positive', 'neutral', 'negative'
            
            # Opsional: Mapping ulang jika ingin format Bahasa Indonesia
            label_map = {
                'positive': 'Positif',
                'neutral': 'Netral',
                'negative': 'Negatif'
            }
            labels.append(label_map.get(label, label))
            scores.append(result['score'])
            
        except Exception as e:
            # Jika ada error pada teks tertentu (jarang terjadi)
            print(f"Error pada teks: {text[:20]}... | {e}")
            labels.append("Netral") # Default fallback
            scores.append(0.0)

    # Masukkan hasil ke DataFrame
    df['sentiment_label'] = labels
    df['confidence_score'] = scores
    
    return df

# def labeling_data(df):
#     """
#     Fungsi untuk memberikan label otomatis berdasarkan skor bintang.
#     Menggunakan logika:
#     1-2 Bintang = Negatif
#     3 Bintang   = Netral
#     4-5 Bintang = Positif
#     """
#     def get_sentiment(score):
#         if score <= 2:
#             return 'Negatif'
#         elif score == 3:
#             return 'Netral'
#         else:
#             return 'Positif'

#     # Terapkan fungsi get_sentiment ke kolom 'score'
#     df['label'] = df['score'].apply(get_sentiment)
    
#     return df

**Main Section**

In [45]:
TARGET_APP = 'com.shopee.id' 
JUMLAH_DATA = 50000          

df_ulasan = scrape_google_play(TARGET_APP, JUMLAH_DATA)

Proses scraping untuk aplikasi: com.shopee.id...
Berhasil mengambil 50000 data mentah.


In [46]:
df_ulasan.head(5)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,fded2b8e-a989-49ef-908e-892b881bdaab,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kesel banget sering masuk tiba tiba ke shoope ...,2,0,NaN,2026-01-27 17:00:19,"Hi kak Rusmini Junita, mohon maaf ya kak atas ...",2026-01-27 18:27:11,NaN
1,4a865d42-843a-41aa-a102-9ddc405587f2,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,kenapa pesanan. cod tidak bisa padahal sudah 2...,2,0,3.67.24,2026-01-27 16:58:58,"Hai Kak eko gaming, maaf Kakak jadi ngga nyama...",2026-01-27 17:25:12,3.67.24
2,3d521416-e8c4-4798-ac7e-f9fc293f45cb,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Ini kenpa ya gk bisa di instal,2,0,NaN,2026-01-27 16:54:06,"Hai kak Abdul Kholik, mohon maaf atas ketidakn...",2026-01-27 17:24:06,NaN
3,28fc4a34-91b8-43c7-a8c4-c4ff60a49cf3,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,titik pengantaran suka berubah sendiri malah b...,2,0,3.66.27,2026-01-27 16:25:56,"Hai kak, maaf ya untuk kendala yang kaka alami...",2026-01-27 17:25:28,3.66.27
4,342abb28-a83e-4aab-930c-06877c7db5a3,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,TOLONG! kalau aku lagi baca Wattpad tuh jangan...,2,0,3.62.40,2026-01-27 14:41:26,"Hai kak Khofifah Nur, mohon maaf atas ketidakn...",2026-01-27 15:17:36,3.62.40


In [47]:
df_ulasan.info()

<class 'pandas.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   reviewId              50000 non-null  str           
 1   userName              50000 non-null  str           
 2   userImage             50000 non-null  str           
 3   content               50000 non-null  str           
 4   score                 50000 non-null  int64         
 5   thumbsUpCount         50000 non-null  int64         
 6   reviewCreatedVersion  38189 non-null  str           
 7   at                    50000 non-null  datetime64[us]
 8   replyContent          38969 non-null  str           
 9   repliedAt             38969 non-null  datetime64[us]
 10  appVersion            38189 non-null  str           
dtypes: datetime64[us](2), int64(2), str(7)
memory usage: 4.2 MB


In [48]:
df_ulasan.describe(include='all')

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
count,50000,50000,50000,50000,50000.0,50000.000000,38189,50000,38969,38969,38189
unique,50000,500,501,47779,NaN,NaN,448,NaN,29209,NaN,448
top,fded2b8e-a989-49ef-908e-892b881bdaab,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Bagus,NaN,NaN,2.95.52,NaN,"hai kak, mohon maaf atas kendala yang kamu ala...",NaN,2.95.52
freq,1,49500,49500,166,NaN,NaN,4232,NaN,107,NaN,4232
mean,NaN,NaN,NaN,NaN,2.0,3.683100,NaN,2024-03-31 23:10:58.940819,NaN,2024-04-13 04:42:16.605609,NaN
min,NaN,NaN,NaN,NaN,2.0,0.000000,NaN,2022-12-24 01:17:50,NaN,2016-02-15 17:24:39,NaN
25%,NaN,NaN,NaN,NaN,2.0,0.000000,NaN,2023-06-02 10:42:00.250000,NaN,2023-06-13 14:12:46,NaN
50%,NaN,NaN,NaN,NaN,2.0,0.000000,NaN,2024-02-02 15:35:03,NaN,2024-03-15 04:25:51,NaN
75%,NaN,NaN,NaN,NaN,2.0,0.000000,NaN,2025-01-08 13:43:06.500000,NaN,2025-02-08 10:33:34,NaN
max,NaN,NaN,NaN,NaN,2.0,11517.000000,NaN,2026-01-27 17:00:19,NaN,2026-01-27 18:27:11,NaN


**Cek Validitas dan Kebersihan Data**

In [49]:
cek_nan(df_ulasan)

,Jumlah Nilai Missing,Persentase Nilai Missing (%)
reviewCreatedVersion,11811,23.622
appVersion,11811,23.622
replyContent,11031,22.062
repliedAt,11031,22.062


In [50]:
visualize_row_with_duplicated(df_ulasan)

Tidak ada data duplikat


In [51]:
df = df_ulasan[['content', 'score']]

In [52]:
visualize_row_with_duplicated(df)

Jumlah Data Duplikat : 2221
Pasangan Data Duplikat :


,content,score
18125,*⁠\⁠0⁠/⁠*,2
32270,*⁠\⁠0⁠/⁠*,2


,content,score
25492,2 dulu,2
33919,2 dulu,2
39817,2 dulu,2
42511,2 dulu,2
46442,2 dulu,2


,content,score
28276,Agak lemot,2
29744,Agak lemot,2
36027,Agak lemot,2
47830,Agak lemot,2


,content,score
7194,Alhamdulillah baik,2
8269,Alhamdulillah baik,2


...

,content,score
11714,😈,2
35422,😈,2


,content,score
31228,😍,2
37755,😍,2


,content,score
25911,😭,2
30396,😭,2


,content,score
6536,🙂,2
42306,🙂,2


,content,score
17369,🙏,2
24977,🙏,2


In [53]:
df.drop_duplicates(inplace=True)

In [54]:
visualize_row_with_duplicated(df)

Tidak ada data duplikat


In [55]:
print(f"Ukuran Data Setelah Pembersihan NaN dan Duplicated: {df.shape}")

Ukuran Data Setelah Pembersihan NaN dan Duplicated: (47779, 2)


**Labeling Data**

In [56]:
print("Labeling data berdasarkan skor bintang")
df_labeled = labeling_data(df)

Labeling data berdasarkan skor bintang


In [57]:
df_labeled.head(5)

,content,score,label
0,kesel banget sering masuk tiba tiba ke shoope ...,2,Negatif
1,kenapa pesanan. cod tidak bisa padahal sudah 2...,2,Negatif
2,Ini kenpa ya gk bisa di instal,2,Negatif
3,titik pengantaran suka berubah sendiri malah b...,2,Negatif
4,TOLONG! kalau aku lagi baca Wattpad tuh jangan...,2,Negatif


**Pemeriksaan Distribusi Data, memastikan apakah terdapat imbalance atau tidak**

In [58]:
print("Distribusi Data per Kelas:")
print(df_labeled['label'].value_counts())

Distribusi Data per Kelas:
label
Negatif    47779
Name: count, dtype: int64


**Konversi ke csv untuk mempermudah dalam pembersihan data lebih lanjut**

In [ ]:
nama_file = 'dataset_ulasan_playstore.csv'
df_labeled.to_csv(nama_file, index=False)

print(f"Selesai! Data berhasil disimpan ke '{nama_file}'")
print(f"Total data: {len(df_labeled)}")

Selesai! Data berhasil disimpan ke 'dataset_ulasan_playstore_negatif.csv'
Total data: 47779
